In [1]:
#Sanity test if larq works as intended

import tensorflow as tf
import larq as lq

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling2D, BatchNormalization, Activation, Flatten
from larq.layers import QuantDense, QuantConv2D

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
          kernel_quantizer="ste_sign",
          kernel_constraint="weight_clip")

input_shape = (train_images.shape[1:])

def build_model():
    
    inputs = Input(shape=input_shape)
    
    # In the first layer we only quantize the weights and not the input
    x = QuantConv2D(32, (3,3),
                    kernel_quantizer="ste_sign",
                    kernel_constraint="weight_clip",
                    use_bias=False)(inputs)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization(scale=False)(x)
    
    x = QuantConv2D(64, (3,3),
                    use_bias=False,
                    **kwargs)(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization(scale=False)(x)
    
    x = QuantConv2D(64, (3,3),
                    use_bias=False,
                    **kwargs)(x)
    x = BatchNormalization(scale=False)(x)
    x = Flatten()(x)
    
    x = QuantDense(64, use_bias=False, **kwargs)(x)
    x = BatchNormalization(scale=False)(x)
    
    x = QuantDense(10, use_bias=False, **kwargs)(x)
    x = BatchNormalization(scale=False)(x)
    outputs = Activation("softmax")(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [4]:
model = build_model()
lq.models.summary(model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
+model stats---------------------------------------------------------------------------------------------------+
| Layer                  Input prec.               Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                              (bit)                            x 1       x 1    (kB)                          |
+--------------------------------------------------------------------------------------------------------------+
| input_1                          -  ((None, 28, 28, 1),)        0         0       0           ?            ? |
| quant_conv2d                     -      (-1, 26, 26, 32)      288         0    0.04           0       194688 |
| max_pooling2d                    -      (-1, 13, 13, 32)        0         0       0           0            0 |
| batch_normalization              -      (-1, 13, 13, 32)        0        64    0.25           0            0 |
| quant_conv2d_

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, 
          batch_size=64, 
          epochs=6,
          validation_data=(test_images, test_labels))

test_loss, test_acc = model.evaluate(test_images, test_labels, batch_size=64)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 10000 samples
Epoch 1/6
60000/60000 [==============================] - 13s 223us/sample - loss: 0.6512 - acc: 0.9091 - val_loss: 0.4517 - val_acc: 0.9634
Epoch 2/6
60000/60000 [==============================] - 9s 157us/sample - loss: 0.4720 - acc: 0.9631 - val_loss: 0.4579 - val_acc: 0.9678
Epoch 3/6
60000/60000 [==============================] - 9s 155us/sample - loss: 0.4476 - acc: 0.9694 - val_loss: 0.4822 - val_acc: 0.9611
Epoch 4/6
60000/60000 [==============================] - 9s 154us/sample - loss: 0.4401 - acc: 0.9715 - val_loss: 0.4748 - val_acc: 0.9643
Epoch 5/6
60000/60000 [==============================] - 9s 154us/sample - loss: 0.4307 - acc: 0.9751 - val_loss: 0.4629 - val_acc: 0.9665
Epoch 6/6
10000/10000 [==============================] - 1s 59us/sample - loss: 0.4102 - acc: 0.9769


In [6]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 97.69 %
